In [0]:
%sh
ls -lrt /databricks-datasets/COVID



In [0]:

print(" hello world")

In [0]:
%sh ls


In [0]:
dbutils.widgets.multiselect("name", "a",["a", "b", "c","d","e"],"enter soame " )
var=dbutils.widgets.get("name")
var_lst=var.split(",")
print(var_lst)
print(type(var))
print(var)


In [0]:
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
#print(rawdf1.schema)
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/default/ethi_volume/custsmodified",mode='permissive')
print("after keeping nulls on the wrong data format",cleandf1.count())#all rows count
display(cleandf1)

print('_______________')
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/default/ethi_volume/custsmodified",mode='dropMalformed')
print("after cleaning wrong data (type mismatch, column number mismatch)",len(cleandf1.collect()))
display(cleandf1)

In [0]:
#Creating rejection dataset to send to our source system for future fix
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True),StructField("corruptedrows",StringType())])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/default/ethi_volume/custsmodified",mode='permissive',columnNameOfCorruptRecord="corruptedrows")
#Create a reject dataset
rejectdf1=cleandf1.where("corruptedrows is not null")
#display(rejectdf1)
rejectdf1.write.csv("/Volumes/workspace/wd36_ethi/ingestion_volume/staging/reject",mode="overwrite",header=True)
retaineddf1=cleandf1.where("corruptedrows is null")
print("Overall rows in the source data is ",len(cleandf1.collect()))
print("Rejected rows in the source data is ",len(rejectdf1.collect()))
print("Clean rows in the source data is ",len(retaineddf1.collect()))

In [0]:
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/workspace/default/ethi_volume/custsmodified/"])
strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv(path=["/Volumes/workspace/default/ethi_volume/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_T*")
display(rawdf1)
display(rawdf2)

In [0]:
cleanseddf=rawdf1.na.drop(how="any")
#This function will drop any column in a given row with null otherwise this function returns rows with no null columns - In a #scenario of if the source send the Datascience Model features (we shouldn't have any one feature with null value, hence we #can use this function)
print("any one row in the raw df with age null")
display(rawdf1.where("age is null"))
print("_____++++HIHI________")
print("any one row in the cleansed df with age null")
display(cleanseddf.where("age is null"))#any one column contains null will be cleaned
print("_____O+++IIBU________")
cleanseddf=rawdf1.na.drop(how="any",subset=["id","age"])#If we need CDE without nulls (Critical Data Elements/Significant columns) columns
print("any one row in the cleansed df with id or age null")
display(cleanseddf)
cleanseddf=rawdf1.na.drop(how="all",subset=["firstname","lastname"])#4000004,Gretchen,,66,
print("any one row in the cleansed df with firstname and lastname is null")
print("Total rows without firstname and lastname with null values",len(cleanseddf.collect()))
display(cleanseddf)#We are taking this DF further for munging..

In [0]:
scrubbeddf1=cleanseddf.na.fill('not provided',subset=["lastname","profession"])#fill will help us replace nulls with some value
display(scrubbeddf1)
find_replace_values_dict1={'Pilot':'Captain','Actor':'Celeberity'}
find_replace_values_dict2={'not provided':'NA'}
scrubbeddf2=scrubbeddf1.na.replace(find_replace_values_dict1,subset=["profession"])#fill function is helping us find and replace the values
scrubbeddf3=scrubbeddf2.na.replace(find_replace_values_dict2,subset=["lastname"])
display(scrubbeddf3)

In [0]:
#display(scrubbeddf3.where("id in ('4000001')"))#before row level dedup
dedupdf1=scrubbeddf3.distinct()#It will remove the row level duplicates
#display(dedupdf1.where("id in ('4000001')"))
#display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
#dedupdf2=dedupdf1.coalesce(1).dropDuplicates(subset=["id"])#It will remove the column level
dedupdf2=dedupdf1
display(dedupdf2.where("id in ('4000003')"))
dedupdf2=dedupdf1.coalesce(1).orderBy(["id","age"],ascending=[True,False]).dropDuplicates(subset=["id"])
#dedupdf2=dedupdf1.coalesce(1).orderBy(["id","age"],ascending=[True,True])
display(dedupdf2.where("id in ('4000003')"))


In [0]:
from pyspark.sql.functions import lit,initcap,col
#withColumn("stringcolumnname to add in the df",lit('hardcoded')/initcap(col("colname")))
standarddf1=dedupdf2.withColumn("sourcesystem",initcap(lit('Ret')))#SparkSQL - DSL(FBP)
display(standarddf1.limit(20))

standarddf2=standarddf1.withColumn("profession",initcap("profession"))



standarddf2.where("id rlike '[a-zA-Z]'").show()#rlike is regular expression like function that help us identify any string data in our DF column
standarddf2.where("age rlike '[^0-9]'").show()#checking for any non number values in age column
#standarddf3=standarddf2.withColumn

In [0]:
from pyspark.sql.functions import regexp_replace,replace
#Let's apply scrubbing features to our id column to replace ten with 10 (or we can think of using GenAI here)
replaceval={'one':'1','two':'2','three':'3','four':'4','five':'5','six':'6','seven':'7','eight':'8','nine':'9','ten':'10'}
standarddf3=standarddf2.na.replace(replaceval,["id"])
#standarddf3=standarddf2.withColumn("id",replace("id",lit('ten'),"10"))
standarddf3=standarddf3.withColumn("age",regexp_replace("age",'-',""))
#display(standarddf3)
standarddf4=standarddf3.withColumn("id",col("id").cast("long"))
standarddf4=standarddf4.withColumn("age",col("age").cast("short"))
standarddf5=standarddf4.withColumnRenamed("id","custid")
standarddf5=standarddf4.withColumnsRenamed({"id":"custid","sourcesystem":"srcsystem"})
display(standarddf5)
standarddf6=standarddf5.select("custid", "age", "firstname","lastname","profession","srcsystem")
#display(standarddf6)
mungeddf=standarddf6
display(mungeddf.take(10))

In [0]:
from pyspark.sql.functions import lit,current_date
derived_datadt="25/09/01"
#enrichdf1=mungeddf.select(lit(derived_datadt),current_date())
#display(enrichdf1)

